<a href="https://colab.research.google.com/github/EYBishal/Python-Training-Bootcamp/blob/main/26th-Dec-2025/Exercise-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [23]:

from google.colab import files
uploaded = files.upload()





Saving orders_large_bad.json to orders_large_bad.json


In [22]:
from pyspark.sql import functions as f
csv_path="/content/orders_large_bad.csv"
df=spark.read.csv(csv_path,header=True,inferSchema=False)
df.printSchema()
df.orderBy(f.rand()).show(20)
df.count()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)

+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|     city|   category|    product| amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|ORD00039312|    C039312|Hyderabad|    Fashion|      Shoes|  59903|2024/01/13|Completed|
|ORD00076392|    C026392|  Chennai|Electronics|     Mobile|  60689|2024-01-13|Completed|
|ORD00123700|    C023700|    Delhi|    Fashion|     TShirt|  26466|2024-02-10|Cancelled|
|ORD00062198|    C012198|  Chennai|    Fashion|      Jeans|  16480|2024-02-08|Completed|
|ORD00172287|    C022287|    Delhi|       Home|AirPurif

300000

In [24]:
json_path="/content/orders_large_bad.json"
df1=spark.read.json(json_path)
df1.printSchema()
df1.orderBy(f.rand()).show(20)
df1.count()

root
 |-- amount: string (nullable = true)
 |-- category: string (nullable = true)
 |-- city: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- product: string (nullable = true)
 |-- status: string (nullable = true)

+------+-----------+---------+-----------+------------+-----------+-------+---------+
|amount|   category|     city|customer_id|  order_date|   order_id|product|   status|
+------+-----------+---------+-----------+------------+-----------+-------+---------+
|  7467|    Grocery|  Chennai|    C049134|invalid_date|ORD00099134|  Sugar|Completed|
| 82914|    Fashion|Hyderabad|    C032523|  04/01/2024|ORD00182523|  Shoes|Completed|
| 23464|    Fashion|Bangalore|    C046635|  2024-02-05|ORD00246635|  Jeans|Completed|
| 14354|Electronics|  Chennai|    C016599|  2024-02-29|ORD00216599| Laptop|Completed|
|      |    Grocery|  Kolkata|    C016327|  2024-01-08|ORD00016327|   Rice|C

300000

In [27]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType
schema=StructType([
    StructField("order_id",StringType(),True),
    StructField("customer_id",StringType(),True),
    StructField("city",StringType(),True),
    StructField("category",StringType(),True),
    StructField("product",StringType(),True),
    StructField("amount",IntegerType(),True),
    StructField("order_date",DateType(),True),
    StructField("status",StringType(),True)
    ])
df_schema=spark.read.csv(csv_path,header=True,schema=schema)
df_schema.printSchema()
df_schema.show(20)




root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- status: string (nullable = true)

+-----------+-----------+-----------+-----------+-----------+------+----------+---------+
|   order_id|customer_id|       city|   category|    product|amount|order_date|   status|
+-----------+-----------+-----------+-----------+-----------+------+----------+---------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |  NULL|      NULL|Cancelled|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar| 35430|2024-01-02|Completed|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile| 65358|2024-01-03|Completed|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|  5558|2024-01-04|Completed|
|ORD00000004|    C000004|       Pune|       Home|

**Schema inference is dangerous at scale because it’s unpredictable, slow, non‑deterministic, and can silently corrupt data—especially in large or messy datasets.**

In [29]:
df_schema.select("city").distinct().count()

7

In [28]:
fix_col=["order_id","customer_id","city","category","product","amount","order_date","status"]
for i in fix_col:
  df_schema=df_schema.withColumn(i,f.lower(f.trim(i)))
df_schema.show(20)

+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|   order_id|customer_id|     city|   category|    product|amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|ord00000000|    c000000|hyderabad|    grocery|        oil|  NULL|      NULL|cancelled|
|ord00000001|    c000001|     pune|    grocery|      sugar| 35430|2024-01-02|completed|
|ord00000002|    c000002|     pune|electronics|     mobile| 65358|2024-01-03|completed|
|ord00000003|    c000003|bangalore|electronics|     laptop|  5558|2024-01-04|completed|
|ord00000004|    c000004|     pune|       home|airpurifier| 33659|2024-01-05|completed|
|ord00000005|    c000005|    delhi|    fashion|      jeans|  8521|2024-01-06|completed|
|ord00000006|    c000006|    delhi|    grocery|      sugar| 42383|2024-01-07|completed|
|ord00000007|    c000007|     pune|    grocery|       rice| 45362|2024-01-08|completed|
|ord00000008|    c000008|bangalo

In [30]:
df_schema.select("city").distinct().count()

7

In [44]:

affected = df_schema.filter(
    (f.col("amount").isNull())
).count()

print("Affected rows during amount cleaning:", affected)


Affected rows during amount cleaning: 0


In [38]:

re_amount=f.regexp_extract(f.col("amount"),r"(\d+)",0)
df_schema=df_schema.withColumn("amount",f.when((re_amount=="")|re_amount.isNull(),f.lit(0)).otherwise(re_amount.cast(IntegerType())))
df_schema.show(20)



+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|   order_id|customer_id|     city|   category|    product|amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+------+----------+---------+
|ord00000000|    c000000|hyderabad|    grocery|        oil|     0|      NULL|cancelled|
|ord00000001|    c000001|     pune|    grocery|      sugar| 35430|2024-01-02|completed|
|ord00000002|    c000002|     pune|electronics|     mobile| 65358|2024-01-03|completed|
|ord00000003|    c000003|bangalore|electronics|     laptop|  5558|2024-01-04|completed|
|ord00000004|    c000004|     pune|       home|airpurifier| 33659|2024-01-05|completed|
|ord00000005|    c000005|    delhi|    fashion|      jeans|  8521|2024-01-06|completed|
|ord00000006|    c000006|    delhi|    grocery|      sugar| 42383|2024-01-07|completed|
|ord00000007|    c000007|     pune|    grocery|       rice| 45362|2024-01-08|completed|
|ord00000008|    c000008|bangalo

In [40]:
df.select("amount").count()

300000

In [45]:
df_schema.filter(f.col("order_date").isNull()).count()

50847

In [50]:
df_schema.select("order_date").orderBy(f.rand()).show(20)

+----------+
|order_date|
+----------+
|2024-02-23|
|      NULL|
|2024-01-12|
|      NULL|
|2024-02-28|
|2024-01-11|
|2024-01-24|
|      NULL|
|2024-02-09|
|2024-02-05|
|2024-02-19|
|2024-02-11|
|2024-02-05|
|      NULL|
|2024-02-20|
|2024-01-07|
|2024-01-08|
|2024-02-03|
|2024-02-08|
|2024-02-09|
+----------+
only showing top 20 rows


In [55]:
df_schema=df_schema.withColumn("order_date_clean",f.coalesce(
    f.col("order_date"),
    f.to_date(f.col("order_date"),"yyyy-MM-dd"),
    f.to_date(f.col("order_date"),"yyyy/MM/dd"),
    f.to_date(f.col("order_date"),"yyyy.MM.dd"),
    f.to_date(f.lit("2025-12-26"),"yyyy-MM-dd")
))
df_schema.show(20)

+-----------+-----------+---------+-----------+-----------+------+----------+---------+----------------+
|   order_id|customer_id|     city|   category|    product|amount|order_date|   status|order_date_clean|
+-----------+-----------+---------+-----------+-----------+------+----------+---------+----------------+
|ord00000000|    c000000|hyderabad|    grocery|        oil|     0|      NULL|cancelled|      2025-12-26|
|ord00000001|    c000001|     pune|    grocery|      sugar| 35430|2024-01-02|completed|      2024-01-02|
|ord00000002|    c000002|     pune|electronics|     mobile| 65358|2024-01-03|completed|      2024-01-03|
|ord00000003|    c000003|bangalore|electronics|     laptop|  5558|2024-01-04|completed|      2024-01-04|
|ord00000004|    c000004|     pune|       home|airpurifier| 33659|2024-01-05|completed|      2024-01-05|
|ord00000005|    c000005|    delhi|    fashion|      jeans|  8521|2024-01-06|completed|      2024-01-06|
|ord00000006|    c000006|    delhi|    grocery|      su

In [57]:
df_schema.count()

300000

In [86]:
df_check=df_schema.drop_duplicates(["order_id"])
df_check.count()
s=df_schema.filter(f.col("status")=="completed")
s.count()
s.show()

+-----------+-----------+---------+-----------+-----------+------+----------+---------+----------------+
|   order_id|customer_id|     city|   category|    product|amount|order_date|   status|order_date_clean|
+-----------+-----------+---------+-----------+-----------+------+----------+---------+----------------+
|ord00000001|    c000001|     pune|    grocery|      sugar| 35430|2024-01-02|completed|      2024-01-02|
|ord00000002|    c000002|     pune|electronics|     mobile| 65358|2024-01-03|completed|      2024-01-03|
|ord00000003|    c000003|bangalore|electronics|     laptop|  5558|2024-01-04|completed|      2024-01-04|
|ord00000004|    c000004|     pune|       home|airpurifier| 33659|2024-01-05|completed|      2024-01-05|
|ord00000005|    c000005|    delhi|    fashion|      jeans|  8521|2024-01-06|completed|      2024-01-06|
|ord00000006|    c000006|    delhi|    grocery|      sugar| 42383|2024-01-07|completed|      2024-01-07|
|ord00000007|    c000007|     pune|    grocery|       r

In [69]:
df_schema.rdd.getNumPartitions()

2

In [83]:
import time
st=time.time()
df_schema.groupBy("city", "category", "status").agg(f.count("*").alias("row_count"))
end=time.time()-st

print(f"Elapsed time: {end:.2f} seconds")

Elapsed time: 0.02 seconds


In [77]:
df_schema.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(order_date_clean, 'coalesce('order_date, 'to_date('order_date, yyyy-MM-dd), 'to_date('order_date, yyyy/MM/dd), 'to_date('order_date, yyyy.MM.dd), 'to_date(2025-12-26, yyyy-MM-dd)), None)]
+- Project [order_id#379, customer_id#380, city#381, category#382, product#383, amount#506, order_date#385, status#386, coalesce(cast(order_date#385 as date), to_date(order_date#385, Some(yyyy-MM-dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy/MM/dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy.MM.dd), Some(Etc/UTC), true), to_date(2025-12-26, Some(yyyy-MM-dd), Some(Etc/UTC), true)) AS order_date_clean#640]
   +- Project [order_id#379, customer_id#380, city#381, category#382, product#383, amount#506, order_date#385, status#386, coalesce(cast(order_date#385 as date), to_date(order_date#385, Some(dd-MM-yyyy), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy-MM-dd), Some(Etc/UTC), true), to_date(order_date#385, 

In [78]:
repart_dp=df_schema.repartition("city")

In [79]:
repart_dp.explain(True)

== Parsed Logical Plan ==
'RepartitionByExpression ['city]
+- Project [order_id#379, customer_id#380, city#381, category#382, product#383, amount#506, order_date#385, status#386, coalesce(cast(order_date#385 as date), to_date(order_date#385, Some(yyyy-MM-dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy/MM/dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy.MM.dd), Some(Etc/UTC), true), to_date(2025-12-26, Some(yyyy-MM-dd), Some(Etc/UTC), true)) AS order_date_clean#677]
   +- Project [order_id#379, customer_id#380, city#381, category#382, product#383, amount#506, order_date#385, status#386, coalesce(cast(order_date#385 as date), to_date(order_date#385, Some(yyyy-MM-dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy/MM/dd), Some(Etc/UTC), true), to_date(order_date#385, Some(yyyy.MM.dd), Some(Etc/UTC), true), to_date(2025-12-26, Some(yyyy-MM-dd), Some(Etc/UTC), true)) AS order_date_clean#640]
      +- Project [order_id#379, customer_id#380, city#381, categor

Calculate total revenue per city
35. Calculate total revenue per category
36. Calculate total revenue per product
37. Identify top 10 products by revenue
38. Calculate average order value per city

In [97]:

from pyspark.sql import functions as F

by_city = (
    s.groupBy("city")
     .agg(F.sum("amount").alias("total_revenue"))
)

by_city.show()

by_product = (
    s.groupBy("product")
     .agg(F.sum("amount").alias("total_revenue"))
)
by_product.show()

by_city_sorted = by_city.orderBy(F.col("total_revenue").desc())
by_city_sorted.show()

avg_order_value_by_city = (
    s.groupBy("city")
     .agg(F.avg("amount").alias("avg_order_value"))
)

avg_order_value_by_city.show()



+---------+-------------+
|city     |total_revenue|
+---------+-------------+
|chennai  |1609789247   |
|delhi    |1618999916   |
|bangalore|1608799093   |
|hyderabad|1622199340   |
|kolkata  |1604056497   |
|mumbai   |1605742096   |
|pune     |1626036535   |
+---------+-------------+

+-----------+-------------+
|product    |total_revenue|
+-----------+-------------+
|shoes      |935339102    |
|vacuum     |948217427    |
|airpurifier|940310123    |
|mobile     |931860576    |
|tablet     |948899999    |
|sugar      |936637000    |
|laptop     |950544295    |
|mixer      |945308026    |
|jeans      |939526127    |
|rice       |942854237    |
|tshirt     |924324943    |
|oil        |951800869    |
+-----------+-------------+

+---------+-------------+
|city     |total_revenue|
+---------+-------------+
|pune     |1626036535   |
|hyderabad|1622199340   |
|delhi    |1618999916   |
|chennai  |1609789247   |
|bangalore|1608799093   |
|mumbai   |1605742096   |
|kolkata  |1604056497   |
+---

In [101]:
from pyspark.sql.window import Window
df_c=s.groupBy("city").agg(f.sum("amount").alias("total_revenue"))
w=Window.orderBy(f.col("total_revenue").desc())
df_c.withColumn("rank",f.rank().over(w)).show()

+---------+-------------+----+
|     city|total_revenue|rank|
+---------+-------------+----+
|     pune|   1626036535|   1|
|hyderabad|   1622199340|   2|
|    delhi|   1618999916|   3|
|  chennai|   1609789247|   4|
|bangalore|   1608799093|   5|
|   mumbai|   1605742096|   6|
|  kolkata|   1604056497|   7|
+---------+-------------+----+



In [103]:
df_c=s.groupBy("category","product").agg(f.sum("amount").alias("total_revenue"))
w=Window.partitionBy("category").orderBy(f.col("total_revenue").desc())
df_c.withColumn("rank",f.rank().over(w)).show()

+-----------+-----------+-------------+----+
|   category|    product|total_revenue|rank|
+-----------+-----------+-------------+----+
|electronics|     laptop|    950544295|   1|
|electronics|     tablet|    948899999|   2|
|electronics|     mobile|    931860576|   3|
|    fashion|      jeans|    939526127|   1|
|    fashion|      shoes|    935339102|   2|
|    fashion|     tshirt|    924324943|   3|
|    grocery|        oil|    951800869|   1|
|    grocery|       rice|    942854237|   2|
|    grocery|      sugar|    936637000|   3|
|       home|     vacuum|    948217427|   1|
|       home|      mixer|    945308026|   2|
|       home|airpurifier|    940310123|   3|
+-----------+-----------+-------------+----+



In [104]:
df_c=s.groupBy("category","product").agg(f.sum("amount").alias("total_revenue"))
w=Window.partitionBy("category").orderBy(f.col("total_revenue").desc())
df_c.withColumn("rank",f.rank().over(w)).filter(f.col("rank")==1).show()

+-----------+-------+-------------+----+
|   category|product|total_revenue|rank|
+-----------+-------+-------------+----+
|electronics| laptop|    950544295|   1|
|    fashion|  jeans|    939526127|   1|
|    grocery|    oil|    951800869|   1|
|       home| vacuum|    948217427|   1|
+-----------+-------+-------------+----+



In [105]:
df_c=s.groupBy("city").agg(f.sum("amount").alias("total_revenue"))
w=Window.orderBy(f.col("total_revenue").desc())
df_c.withColumn("rank",f.rank().over(w)).filter(f.col("rank")<=3).show()

+---------+-------------+----+
|     city|total_revenue|rank|
+---------+-------------+----+
|     pune|   1626036535|   1|
|hyderabad|   1622199340|   2|
|    delhi|   1618999916|   3|
+---------+-------------+----+



In [111]:

from pyspark import StorageLevel
from pyspark.sql import functions as f

by_city = s.groupBy("city").agg(f.sum("amount").alias("total_revenue"))
df_c = s.groupBy("category", "product").agg(f.sum("amount").alias("total_revenue"))

df_c_cached = df_c.cache()
df_c_cached.count()


12

In [109]:

import time

t = time.time()
s.groupBy("city").sum("amount").show()
s.groupBy("product").avg("amount").show()
print(f"Elapsed time (no cache): {time.time() - t:.2f} seconds")

s.cache()
g = time.time()
s.groupBy("city").sum("amount").show()
s.groupBy("product").avg("amount").show()
print(f"Elapsed time (cached): {time.time() - g:.2f} seconds")


+---------+-----------+
|     city|sum(amount)|
+---------+-----------+
|  chennai| 1609789247|
|    delhi| 1618999916|
|bangalore| 1608799093|
|hyderabad| 1622199340|
|  kolkata| 1604056497|
|   mumbai| 1605742096|
|     pune| 1626036535|
+---------+-----------+

+-----------+------------------+
|    product|       avg(amount)|
+-----------+------------------+
|      shoes| 39450.80357670083|
|     vacuum| 39842.74242615236|
|airpurifier|39615.357389619145|
|     mobile|39608.134313767165|
|     tablet| 39681.34483335424|
|      sugar| 39836.55154814563|
|     laptop| 39688.69707724426|
|      mixer|39575.819559574644|
|      jeans|  39502.4439539186|
|       rice| 39435.11803086704|
|     tshirt| 39492.62734458449|
|        oil| 39874.35563468789|
+-----------+------------------+

Elapsed time (no cache): 3.28 seconds
+---------+-----------+
|     city|sum(amount)|
+---------+-----------+
|  chennai| 1609789247|
|    delhi| 1618999916|
|bangalore| 1608799093|
|hyderabad| 1622199340|


In [110]:
s.unpersist()

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: int, order_date: string, status: string, order_date_clean: date]

**Over-caching wastes memory, causes GC pressure, spills, OOM**

In [112]:
s.write.mode("overwrite").parquet("parquet")

In [113]:
s.write.partitionBy("city").mode("overwrite").parquet("/parquet_by_city")

In [114]:
c=s.groupBy("product").avg("amount")
c.write.mode("overwrite").orc("/orc_by_product")

In [116]:

op = spark.read.parquet("/parquet_by_city")
co = spark.read.format("orc").load("/orc_by_product")
op.printSchema()
co.printSchema()


root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date_clean: date (nullable = true)
 |-- city: string (nullable = true)

root
 |-- product: string (nullable = true)
 |-- avg(amount): double (nullable = true)



**f = df.filter(df.amount > 50000).show()**

.show() is an action that returns None (prints rows, doesn’t return a DataFrame).
Assigning it back makes df = None, so any later transformation fails.

In [ ]:

x = df.groupBy("city").count().show()
x.select("city").show()
#shows none error

In [118]:
s.orderBy("amount").show()
s.explain(True)

+-----------+-----------+---------+-----------+-------+------+----------+---------+----------------+
|   order_id|customer_id|     city|   category|product|amount|order_date|   status|order_date_clean|
+-----------+-----------+---------+-----------+-------+------+----------+---------+----------------+
|ord00180136|    c030136|hyderabad|    fashion|  shoes|     0|      NULL|completed|      2025-12-26|
|ord00000171|    c000171|hyderabad|electronics| mobile|     0|2024-02-21|completed|      2024-02-21|
|ord00180139|    c030139|   mumbai|    grocery|    oil|     0|2024-01-20|completed|      2024-01-20|
|ord00000322|    c000322|bangalore|       home| vacuum|     0|2024-01-23|completed|      2024-01-23|
|ord00180063|    c030063|     pune|    grocery|    oil|     0|      NULL|completed|      2025-12-26|
|ord00000174|    c000174|bangalore|    grocery|    oil|     0|2024-02-24|completed|      2024-02-24|
|ord00180148|    c030148|  kolkata|    grocery|    oil|     0|2024-01-29|completed|      20

**Debug a slow Spark job**

Plan: df.explain("formatted"); spot shuffles/joins.
Spark UI: Check skew, spills, GC, long tasks.
AQE: spark.conf.set("spark.sql.adaptive.enabled","true").
Joins: Broadcast small dims; filter early; select needed columns.
Partitions: Tune spark.sql.shuffle.partitions; avoid global orderBy.

In [119]:

critical_cols = ["order_id","city","product","amount"]

for c in critical_cols:
    print(c, s.filter(s[c].isNull()).count())


order_id 0
city 0
product 0
amount 0


In [120]:
print(s.dtypes)

[('order_id', 'string'), ('customer_id', 'string'), ('city', 'string'), ('category', 'string'), ('product', 'string'), ('amount', 'int'), ('order_date', 'string'), ('status', 'string'), ('order_date_clean', 'date')]


In [121]:
print("Final count:", s.count())

Final count: 285000


**Used Parquet with partitioning to reduce I/O and enable partition**
**Cached reused DataFrames (by_city, df_c) to avoid repeated shuffles.**
**Enabled AQE + broadcast joins to optimize large joins and reduce shuffle costs.**